tgb - 11/27/2019 - Training the network for data that was vertically-interpolated on the eps grid

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
import tensorflow.math as tfm
import tensorflow_probability as tfp
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog


W1205 09:15:24.734439 22712538859328 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:145: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1205 09:15:24.734985 22712538859328 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:148: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

W1205 09:15:24.735433 22712538859328 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:148: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM


# 1) Design NN with RH_eps and T_eps as inputs

## 1.1) Rescaling

In [2]:
scale_dict = {
    'PHQEPS': L_V/G,  
    'TPHYSTNDEPS': C_P/G, 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
}

For quick test, use equal pressure thickness for all eps layers (equal weighting in eps space)

In [3]:
PS = 1e5
eps_res = 1e2
dP = PS/eps_res

In [4]:
for v in ['PHQEPS','TPHYSTNDEPS']:
    scale_dict[v] *= dP

In [5]:
save_pickle('./nn_config/scale_dicts/127_POG_scaling.pkl', scale_dict)

## 1.2) Build data generator

In [5]:
in_vars = ['RHEPS','TBPEPS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQEPS','TPHYSTNDEPS','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [6]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [7]:
X, Y = valid_gen[12]; X.shape, Y.shape

((1024, 204), (1024, 204))

Reduce number of layers to 5 since less data and wider layers

In [8]:
inp = Input(shape=(204,))
densout = Dense(256, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (3):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(204, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
NNmodel = tf.keras.models.Model(inp,out)

W1128 07:43:50.146628 23269534324544 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
NNmodel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 204)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               52480     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0     

In [13]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG131.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [14]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

In [15]:
Nep = 20
NNmodel.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
3448/3448 [==============================] - 44s 13ms/step - loss: 31.9895 - val_loss: 32.5142
Epoch 2/20
3448/3448 [==============================] - 43s 12ms/step - loss: 31.6992 - val_loss: 32.5529
Epoch 3/20
3448/3448 [==============================] - 43s 13ms/step - loss: 31.4743 - val_loss: 31.9120
Epoch 4/20
3448/3448 [==============================] - 45s 13ms/step - loss: 31.2693 - val_loss: 31.7280
Epoch 5/20
3448/3448 [==============================] - 45s 13ms/step - loss: 31.1017 - val_loss: 31.7779
Epoch 6/20
3448/3448 [==============================] - 47s 14ms/step - loss: 30.9046 - val_loss: 31.7368
Epoch 7/20
3448/3448 [==============================] - 45s 13ms/step - loss: 30.7238 - val_loss: 31.3456
Epoch 8/20
3448/3448 [==============================] - 45s 13ms/step - loss: 30.5716 - val_loss: 32.7376
Epoch 9/20
3448/3448 [==============================] - 48s 14ms/step - loss: 30.4611 - val_loss: 31.3613
Epoch 10/20
3448/3448 [=======================

# 2) Repeat with larger dataset

In [6]:
in_vars = ['RHEPS','TBPEPS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQEPS','TPHYSTNDEPS','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/132_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [7]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/132_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/131_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [8]:
X, Y = valid_gen[19]; X.shape, Y.shape

((1024, 204), (1024, 204))

In [9]:
inp2 = Input(shape=(204,))
densout = Dense(256, activation='linear')(inp2)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(204, activation='linear')(densout)
out2 = LeakyReLU(alpha=0.3)(densout)
NNmodel2 = tf.keras.models.Model(inp2,out2)

W1128 20:34:22.045083 22889390782272 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
NNmodel2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 204)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               52480     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0     

In [11]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG132.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [12]:
NNmodel2.compile(tf.keras.optimizers.Adam(),loss=mse)

In [14]:
Nep = 20
NNmodel2.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
18336/18336 [==============================] - 324s 18ms/step - loss: 41.6340 - val_loss: 35.3406
Epoch 2/20
18336/18336 [==============================] - 234s 13ms/step - loss: 32.7102 - val_loss: 33.4377
Epoch 3/20
18336/18336 [==============================] - 235s 13ms/step - loss: 31.3168 - val_loss: 32.3228
Epoch 4/20
18336/18336 [==============================] - 240s 13ms/step - loss: 30.5962 - val_loss: 30.9279
Epoch 5/20
18336/18336 [==============================] - 239s 13ms/step - loss: 30.1226 - val_loss: 30.9347
Epoch 6/20
18336/18336 [==============================] - 235s 13ms/step - loss: 29.7778 - val_loss: 30.4701
Epoch 7/20
18336/18336 [==============================] - 237s 13ms/step - loss: 29.4979 - val_loss: 30.9430
Epoch 8/20
18336/18336 [==============================] - 237s 13ms/step - loss: 29.2758 - val_loss: 29.9549
Epoch 9/20
18336/18336 [==============================] - 237s 13ms/step - loss: 29.0793 - val_loss: 30.0082
Epoch 10/20
18336/1

# 3) Repeat with only 30 levels

## 3.1) Preprocessing

In [2]:
#!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/134_POG_RHTepsinput_dQdtdTdtepsotuput_test30lev_June.yml

In [4]:
scale_dict = {
    'PHQEPS': L_V/G,  
    'TPHYSTNDEPS': C_P/G, 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
}

In [5]:
PS = 1e5
eps_res = 30
dP = PS/eps_res

In [6]:
for v in ['PHQEPS','TPHYSTNDEPS']:
    scale_dict[v] *= dP

In [7]:
save_pickle('./nn_config/scale_dicts/134_POG_scaling.pkl', scale_dict)

## 3.2) Data generator

In [8]:
in_vars = ['RHEPS','TBPEPS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQEPS','TPHYSTNDEPS','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/134_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/134_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [9]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/134_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/134_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [10]:
X, Y = valid_gen[19]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 3.3) Model and training

In [11]:
inp = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (3):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
NNmodel = tf.keras.models.Model(inp,out)

W1130 20:55:00.279494 22966316377920 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
NNmodel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0     

In [13]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG134.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [14]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

In [16]:
Nep = 20
NNmodel.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
3448/3448 [==============================] - 22s 6ms/step - loss: 293.5796 - val_loss: 295.9517
Epoch 2/20
3448/3448 [==============================] - 24s 7ms/step - loss: 292.7631 - val_loss: 299.7634
Epoch 3/20
3448/3448 [==============================] - 22s 6ms/step - loss: 292.2214 - val_loss: 292.8768
Epoch 4/20
3448/3448 [==============================] - 22s 6ms/step - loss: 291.7000 - val_loss: 292.6278
Epoch 5/20
3448/3448 [==============================] - 22s 6ms/step - loss: 291.1524 - val_loss: 291.6248
Epoch 6/20
3448/3448 [==============================] - 22s 6ms/step - loss: 290.5519 - val_loss: 295.2890
Epoch 7/20
3448/3448 [==============================] - 22s 6ms/step - loss: 289.9311 - val_loss: 295.7925
Epoch 8/20
3448/3448 [==============================] - 22s 6ms/step - loss: 289.6223 - val_loss: 293.3192
Epoch 9/20
3448/3448 [==============================] - 25s 7ms/step - loss: 288.9978 - val_loss: 292.0082
Epoch 10/20
3448/3448 [==============

# 4) Repeat with dropout

In [19]:
inp2 = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp2)
densout = LeakyReLU(alpha=0.3)(densout)
densout = Dropout(rate=0.01)(densout)
for i in range (3):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
    densout = Dropout(rate=0.01)(densout)
densout = Dense(64, activation='linear')(densout)
densout = Dropout(rate=0.01)(densout)
out2 = LeakyReLU(alpha=0.3)(densout)
NNmodel2 = tf.keras.models.Model(inp2,out2)

In [20]:
NNmodel2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               16640     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0   

In [21]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG134_Dropout.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [22]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

In [23]:
Nep = 20
NNmodel.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
3448/3448 [==============================] - 39s 11ms/step - loss: 465.9679 - val_loss: 350.4284
Epoch 2/20
3448/3448 [==============================] - 41s 12ms/step - loss: 352.5980 - val_loss: 332.3417
Epoch 3/20
3448/3448 [==============================] - 38s 11ms/step - loss: 337.8967 - val_loss: 320.9518
Epoch 4/20
3448/3448 [==============================] - 42s 12ms/step - loss: 328.3691 - val_loss: 314.6974
Epoch 5/20
3448/3448 [==============================] - 40s 11ms/step - loss: 322.3559 - val_loss: 310.3371
Epoch 6/20
3448/3448 [==============================] - 40s 12ms/step - loss: 317.7420 - val_loss: 310.4449
Epoch 7/20
3448/3448 [==============================] - 42s 12ms/step - loss: 314.4533 - val_loss: 306.2188
Epoch 8/20
3448/3448 [==============================] - 38s 11ms/step - loss: 311.7743 - val_loss: 304.1707
Epoch 9/20
3448/3448 [==============================] - 41s 12ms/step - loss: 309.3662 - val_loss: 302.4541
Epoch 10/20
3448/3448 [=====

# 5) Before increasing dataset for eps 30lev, try small dataset for p 30lev

In [28]:
scale_dict = load_pickle('./nn_config/scale_dicts/100_POG_scaling.pkl')

In [30]:
in_vars = ['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/103_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/103_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [31]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/103_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/103_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [32]:
X, Y = valid_gen[19]; X.shape, Y.shape

((1024, 64), (1024, 64))

In [33]:
inp5 = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp5)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (3):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out5 = LeakyReLU(alpha=0.3)(densout)
NNmodel5 = tf.keras.models.Model(inp5,out5)

In [34]:
NNmodel5.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               16640     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 256)               0   

In [35]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG104_Small.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [36]:
NNmodel5.compile(tf.keras.optimizers.Adam(),loss=mse)

In [37]:
Nep = 20
NNmodel5.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
3448/3448 [==============================] - 33s 9ms/step - loss: 783.4452 - val_loss: 646.2542
Epoch 2/20
3448/3448 [==============================] - 28s 8ms/step - loss: 626.3573 - val_loss: 615.3320
Epoch 3/20
3448/3448 [==============================] - 39s 11ms/step - loss: 603.5838 - val_loss: 599.2223
Epoch 4/20
3448/3448 [==============================] - 27s 8ms/step - loss: 588.1144 - val_loss: 586.5352
Epoch 5/20
3448/3448 [==============================] - 24s 7ms/step - loss: 577.7999 - val_loss: 585.0277
Epoch 6/20
3448/3448 [==============================] - 24s 7ms/step - loss: 569.8410 - val_loss: 577.7494
Epoch 7/20
3448/3448 [==============================] - 24s 7ms/step - loss: 563.2021 - val_loss: 572.8820
Epoch 8/20
3448/3448 [==============================] - 31s 9ms/step - loss: 558.2204 - val_loss: 568.6889
Epoch 9/20
3448/3448 [==============================] - 39s 11ms/step - loss: 554.1868 - val_loss: 569.0652
Epoch 10/20
3448/3448 [============

# 6) Train eps 30lev with increased dataset

## 6.1) Generate data

In [5]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/136_POG_84G_RHTepsinput_30lev.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [6]:
scale_dict = load_pickle('./nn_config/scale_dicts/134_POG_scaling.pkl')

In [7]:
in_vars = ['RHEPS','TBPEPS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQEPS','TPHYSTNDEPS','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/136_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/134_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [8]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/136_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/134_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [9]:
X, Y = valid_gen[25]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 6.2) Model without dropout

In [10]:
inp = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
NNmodel = tf.keras.models.Model(inp,out)

W1201 13:10:55.142812 23425363851072 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
NNmodel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0     

In [12]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG136.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [13]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

In [14]:
Nep = 20
NNmodel.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
13728/13728 [==============================] - 99s 7ms/step - loss: 351.9154 - val_loss: 317.7195
Epoch 2/20
13728/13728 [==============================] - 97s 7ms/step - loss: 303.0691 - val_loss: 304.3492
Epoch 3/20
13728/13728 [==============================] - 96s 7ms/step - loss: 294.5087 - val_loss: 298.8146
Epoch 4/20
13728/13728 [==============================] - 96s 7ms/step - loss: 290.1831 - val_loss: 296.5876
Epoch 5/20
13728/13728 [==============================] - 97s 7ms/step - loss: 287.4218 - val_loss: 293.9257
Epoch 6/20
13728/13728 [==============================] - 97s 7ms/step - loss: 285.4558 - val_loss: 292.1698
Epoch 7/20
13728/13728 [==============================] - 97s 7ms/step - loss: 283.8355 - val_loss: 291.8759
Epoch 8/20
13728/13728 [==============================] - 98s 7ms/step - loss: 282.3604 - val_loss: 292.4342
Epoch 9/20
13728/13728 [==============================] - 96s 7ms/step - loss: 281.2009 - val_loss: 290.4176
Epoch 10/20
13728/1

## 6.3) Model with dropout

In [15]:
inp2 = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp2)
densout = LeakyReLU(alpha=0.3)(densout)
densout = Dropout(rate=0.01)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
    densout = Dropout(rate=0.01)(densout)
densout = Dense(64, activation='linear')(densout)
densout = Dropout(rate=0.01)(densout)
out2 = LeakyReLU(alpha=0.3)(densout)
NNmodel2 = tf.keras.models.Model(inp2,out2)

In [16]:
NNmodel2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               16640     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0   

In [17]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG136_Dropout.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [18]:
NNmodel2.compile(tf.keras.optimizers.Adam(),loss=mse)

In [19]:
Nep = 20
NNmodel2.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
13728/13728 [==============================] - 110s 8ms/step - loss: 408.8735 - val_loss: 317.4173
Epoch 2/20
13728/13728 [==============================] - 108s 8ms/step - loss: 360.3003 - val_loss: 304.5859
Epoch 3/20
13728/13728 [==============================] - 108s 8ms/step - loss: 352.3446 - val_loss: 299.8754
Epoch 4/20
13728/13728 [==============================] - 109s 8ms/step - loss: 348.0168 - val_loss: 296.7669
Epoch 5/20
13728/13728 [==============================] - 108s 8ms/step - loss: 345.3047 - val_loss: 296.6124
Epoch 6/20
13728/13728 [==============================] - 110s 8ms/step - loss: 343.6776 - val_loss: 297.5734
Epoch 7/20
13728/13728 [==============================] - 109s 8ms/step - loss: 342.0906 - val_loss: 295.3697
Epoch 8/20
13728/13728 [==============================] - 109s 8ms/step - loss: 340.8755 - val_loss: 294.3304
Epoch 9/20
13728/13728 [==============================] - 108s 8ms/step - loss: 340.0512 - val_loss: 295.4110
Epoch 10/2

In [20]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/137_POG_84G_p4K_RHTepsinput_30lev.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

# 7) Train eps RH,(T-Ts) 30lev

## 7.1) Without dropout

In [22]:
scale_dict = load_pickle('./nn_config/scale_dicts/134_POG_scaling.pkl')

In [24]:
in_vars = ['RHEPS','TfromTSEPS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQEPS','TPHYSTNDEPS','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/139_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/138_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [25]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/139_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/138_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [26]:
X, Y = valid_gen[9]; X.shape, Y.shape

((1024, 64), (1024, 64))

In [27]:
inp = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
NNmodel = tf.keras.models.Model(inp,out)

In [28]:
NNmodel.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               16640     
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 256)               0   

In [29]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG139.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [30]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

In [31]:
Nep = 20
NNmodel2.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
13728/13728 [==============================] - 162s 12ms/step - loss: 337.2676 - val_loss: 292.2296
Epoch 2/20
13728/13728 [==============================] - 160s 12ms/step - loss: 334.8176 - val_loss: 287.4976
Epoch 3/20
13728/13728 [==============================] - 159s 12ms/step - loss: 333.9356 - val_loss: 288.3272
Epoch 4/20
13728/13728 [==============================] - 156s 11ms/step - loss: 333.6413 - val_loss: 289.9234
Epoch 5/20
13728/13728 [==============================] - 159s 12ms/step - loss: 332.9615 - val_loss: 288.4107
Epoch 6/20
13728/13728 [==============================] - 159s 12ms/step - loss: 332.6279 - val_loss: 289.2598
Epoch 7/20
13728/13728 [==============================] - 160s 12ms/step - loss: 332.0529 - val_loss: 289.2730
Epoch 8/20
13728/13728 [==============================] - 161s 12ms/step - loss: 331.9261 - val_loss: 290.7738
Epoch 9/20
13728/13728 [==============================] - 163s 12ms/step - loss: 331.5786 - val_loss: 288.2428
E

## 7.2) With dropout

In [32]:
inp2 = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp2)
densout = LeakyReLU(alpha=0.3)(densout)
densout = Dropout(rate=0.01)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
    densout = Dropout(rate=0.01)(densout)
densout = Dense(64, activation='linear')(densout)
densout = Dropout(rate=0.01)(densout)
out2 = LeakyReLU(alpha=0.3)(densout)
NNmodel2 = tf.keras.models.Model(inp2,out2)

In [33]:
NNmodel2.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_21 (Dense)             (None, 256)               16640     
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0   

In [34]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG139_Dropout.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [35]:
NNmodel2.compile(tf.keras.optimizers.Adam(),loss=mse)

In [36]:
Nep = 20
NNmodel2.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
13728/13728 [==============================] - 167s 12ms/step - loss: 406.1932 - val_loss: 319.8354
Epoch 2/20
13728/13728 [==============================] - 161s 12ms/step - loss: 357.5996 - val_loss: 303.8901
Epoch 3/20
13728/13728 [==============================] - 164s 12ms/step - loss: 350.3048 - val_loss: 301.3602
Epoch 4/20
13728/13728 [==============================] - 165s 12ms/step - loss: 345.7416 - val_loss: 294.6618
Epoch 5/20
13728/13728 [==============================] - 162s 12ms/step - loss: 342.9153 - val_loss: 294.8703
Epoch 6/20
13728/13728 [==============================] - 162s 12ms/step - loss: 341.1751 - val_loss: 292.1221
Epoch 7/20
13728/13728 [==============================] - 161s 12ms/step - loss: 339.4255 - val_loss: 292.2285
Epoch 8/20
13728/13728 [==============================] - 162s 12ms/step - loss: 338.7682 - val_loss: 290.2843
Epoch 9/20
13728/13728 [==============================] - 163s 12ms/step - loss: 337.3511 - val_loss: 296.0719
E

# 8) Train p RH,T to dRH/dt, dT/dt 30 lev

## 8.1) Without dropout

In [3]:
scale_dict = {
    'dRHdt': 0.01*L_V/G, # Arbitrary 0.1 factor as specific humidity is generally below 2%  
    'TPHYSTND': C_P/G, 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
}

In [4]:
# Takes representative value for PS since purpose is normalization
PS = 1e5; P0 = 1e5;
P = P0*hyai+PS*hybi; # Total pressure [Pa]
dP = P[1:]-P[:-1]; # Differential pressure [Pa]

In [5]:
for v in ['dRHdt','TPHYSTND']:
    scale_dict[v] *= dP

In [6]:
# Reduce humidity tendencies above level 10
scale_dict['dRHdt'][:11] *= 0.1

In [7]:
save_pickle('./nn_config/scale_dicts/142_POG_scaling.pkl', scale_dict)

In [8]:
#!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/142_POG_84G_RHinput_dRHoutput.yml

In [ ]:
in_vars = ['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['dRHdt','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/142_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/141_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [ ]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/142_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/141_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [11]:
X, Y = valid_gen[19]; X.shape, Y.shape

((1024, 64), (1024, 64))

In [12]:
inp = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
NNmodel = tf.keras.models.Model(inp,out)

W1202 10:23:04.582470 22842140297024 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
NNmodel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0     

In [14]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG142_Dropout.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [15]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

In [16]:
Nep = 20
NNmodel.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
13728/13728 [==============================] - 103s 8ms/step - loss: 3907.3301 - val_loss: 3477.9009
Epoch 2/20
13728/13728 [==============================] - 102s 7ms/step - loss: 3455.0785 - val_loss: 3347.8832
Epoch 3/20
13728/13728 [==============================] - 102s 7ms/step - loss: 3352.1922 - val_loss: 3284.8219
Epoch 4/20
13728/13728 [==============================] - 102s 7ms/step - loss: 3291.3112 - val_loss: 3256.3507
Epoch 5/20
13728/13728 [==============================] - 102s 7ms/step - loss: 3251.5975 - val_loss: 3211.0658
Epoch 6/20
13728/13728 [==============================] - 103s 7ms/step - loss: 3223.8883 - val_loss: 3197.5396
Epoch 7/20
13728/13728 [==============================] - 102s 7ms/step - loss: 3202.2690 - val_loss: 3186.6846
Epoch 8/20
13728/13728 [==============================] - 102s 7ms/step - loss: 3185.2506 - val_loss: 3186.9775
Epoch 9/20
13728/13728 [==============================] - 103s 7ms/step - loss: 3169.9252 - val_loss: 31

In [5]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/143_POG_84G_p4K_RHinput_dRHoutput.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

## 8.2) With dropout

In [ ]:
scale_dict = load_pickle('./nn_config/scale_dicts/142_POG_scaling.pkl')

In [ ]:
inp3 = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp3)
densout = LeakyReLU(alpha=0.3)(densout)
densout = Dropout(rate=0.01)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
    densout = Dropout(rate=0.01)(densout)
densout = Dense(64, activation='linear')(densout)
densout = Dropout(rate=0.01)(densout)
out3 = LeakyReLU(alpha=0.3)(densout)
NNmodel3 = tf.keras.models.Model(inp3,out3)

In [ ]:
NNmodel3.summary()

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG142_DropoutYES.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
NNmodel3.compile(tf.keras.optimizers.Adam(),loss=mse)

In [ ]:
Nep = 20
NNmodel3.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

# 9) Train eps RH,T-Ts to dRH/dt, dT/dt 30 lev

## 9.1) Pre-processing

In [4]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/146_POG_84G_p4K_RHTfromTSepsinput_dRHdtoutput_30lev.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

E1202 19:11:59.294061 22754576410432 ultratb.py:155] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/IPython/utils/_process_posix.py", line 161, in system
    res_idx = child.expect_list(patterns, self.read_timeout)
  File "/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/pexpect/spawnbase.py", line 369, in expect_list
    return exp.expect_loop(timeout)
  File "/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/pexpect/expect.py", line 111, in expect_loop
    incoming = spawn.read_nonblocking(spawn.maxread, timeout)
  File "/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/pexpect/pty_spawn.py", line 499, in read_nonblocking
    if (timeout != 0) and select(timeout):
  File "/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/pexpect/pty_spawn.py", line 449, in select
    return select_ignore_interrupts([self.child_fd], [], [], timeout)[0]
  File "/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/pexpect/utils.py", line 143, in select_ignore

TypeError: can only concatenate str (not "list") to str

In [3]:
#!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/145_POG_84G_RHTfromTSepsinput_dRHdtoutput_30lev.yml

In [4]:
scale_dict = {
    'dRHdtEPS': 0.01*L_V/G,  
    'TPHYSTNDEPS': C_P/G, 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
}

In [5]:
PS = 1e5
eps_res = 30
dP = PS/eps_res

In [6]:
for v in ['dRHdtEPS','TPHYSTNDEPS']:
    scale_dict[v] *= dP*np.ones((30,))

In [7]:
# Reduce humidity tendencies above level 20
scale_dict['dRHdtEPS'][20:] *= 0.1

In [8]:
save_pickle('./nn_config/scale_dicts/144_POG_scaling.pkl', scale_dict)

In [9]:
in_vars = ['RHEPS','TfromTSEPS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['dRHdtEPS','TPHYSTNDEPS','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/145_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/144_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [10]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/145_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/144_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [11]:
X, Y = valid_gen[12]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 9.2) Without dropout

In [12]:
inp2 = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp2)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out2 = LeakyReLU(alpha=0.3)(densout)
NNmodel2 = tf.keras.models.Model(inp2,out2)

W1202 17:45:12.020156 22985903331136 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
NNmodel2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0     

In [14]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG144.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [15]:
NNmodel2.compile(tf.keras.optimizers.Adam(),loss=mse)

In [16]:
Nep = 20
NNmodel2.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
13728/13728 [==============================] - 96s 7ms/step - loss: 1664.8475 - val_loss: 1482.9146
Epoch 2/20
13728/13728 [==============================] - 95s 7ms/step - loss: 1467.2349 - val_loss: 1428.6084
Epoch 3/20
13728/13728 [==============================] - 95s 7ms/step - loss: 1427.2517 - val_loss: 1408.4057
Epoch 4/20
13728/13728 [==============================] - 95s 7ms/step - loss: 1405.3470 - val_loss: 1395.8530
Epoch 5/20
13728/13728 [==============================] - 95s 7ms/step - loss: 1390.5816 - val_loss: 1385.9915
Epoch 6/20
13728/13728 [==============================] - 95s 7ms/step - loss: 1380.1575 - val_loss: 1384.6017
Epoch 7/20
13728/13728 [==============================] - 95s 7ms/step - loss: 1371.1753 - val_loss: 1359.9962
Epoch 8/20
13728/13728 [==============================] - 95s 7ms/step - loss: 1364.1803 - val_loss: 1358.0853
Epoch 9/20
13728/13728 [==============================] - 94s 7ms/step - loss: 1358.2819 - val_loss: 1358.0718
E

# 10) Train p RH,T to PHQflux and TPHYSTNDflux

In [3]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/147_POG_RHinput_FLUXoutput_test.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [4]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/148_POG_84G_RHinput_FLUXoutput.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [5]:
scale_dict = {
    'PHQFLUX': 100,  
    'TPHYSTNDFLUX': 100, 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
}

In [6]:
save_pickle('./nn_config/scale_dicts/148_POG_scaling.pkl', scale_dict)

In [7]:
in_vars = ['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQFLUX','TPHYSTNDFLUX','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/148_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/147_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [8]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/148_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/147_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [9]:
X, Y = valid_gen[12]; X.shape, Y.shape

((1024, 64), (1024, 64))

In [10]:
inp2 = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp2)
densout = LeakyReLU(alpha=0.3)(densout)
densout = Dropout(rate=0.01)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
    densout = Dropout(rate=0.01)(densout)
densout = Dense(64, activation='linear')(densout)
densout = Dropout(rate=0.01)(densout)
out2 = LeakyReLU(alpha=0.3)(densout)
NNmodel2 = tf.keras.models.Model(inp2,out2)

W1205 09:35:59.459128 22712538859328 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
NNmodel2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0     

In [12]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG148.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [13]:
NNmodel2.compile(tf.keras.optimizers.RMSprop(),loss=mse)

In [14]:
Nep = 20
NNmodel2.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
13728/13728 [==============================] - 107s 8ms/step - loss: 667.1096 - val_loss: 561.5792
Epoch 2/20
13728/13728 [==============================] - 105s 8ms/step - loss: 578.5642 - val_loss: 543.7322
Epoch 3/20
13728/13728 [==============================] - 106s 8ms/step - loss: 561.5372 - val_loss: 481.6702
Epoch 4/20
13728/13728 [==============================] - 106s 8ms/step - loss: 551.7746 - val_loss: 506.9128
Epoch 5/20
13728/13728 [==============================] - 106s 8ms/step - loss: 546.0016 - val_loss: 476.5985
Epoch 6/20
13728/13728 [==============================] - 105s 8ms/step - loss: 541.3156 - val_loss: 480.7034
Epoch 7/20
13728/13728 [==============================] - 105s 8ms/step - loss: 538.3044 - val_loss: 499.2775
Epoch 8/20
13728/13728 [==============================] - 106s 8ms/step - loss: 535.8211 - val_loss: 473.8935
Epoch 9/20
13728/13728 [==============================] - 106s 8ms/step - loss: 533.1369 - val_loss: 461.8422
Epoch 10/2

In [15]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/149_POG_84G_p4K_RHinput_FLUXoutput.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

# 11) Train eps RH,T to PHQflux and TPHYSTNDflux

In [16]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/151_POG_84G_RHinput_FLUXoutput_EPS.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [17]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/150_POG_RHinput_FLUXoutput_EPStest.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [18]:
scale_dict = {
    'PHQFLUXEPS': 100,  
    'TPHYSTNDFLUXEPS': 100, 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
}

In [19]:
save_pickle('./nn_config/scale_dicts/151_POG_scaling.pkl', scale_dict)

In [20]:
in_vars = ['RHEPS','TBPEPS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQFLUXEPS','TPHYSTNDFLUXEPS','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/151_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/150_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [21]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/151_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/150_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [22]:
X, Y = valid_gen[5]; X.shape, Y.shape

((1024, 64), (1024, 64))

In [23]:
inp = Input(shape=(64,))
densout = Dense(256, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
densout = Dropout(rate=0.01)(densout)
for i in range (5):
    densout = Dense(256, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
    densout = Dropout(rate=0.01)(densout)
densout = Dense(64, activation='linear')(densout)
densout = Dropout(rate=0.01)(densout)
out = LeakyReLU(alpha=0.3)(densout)
NNmodel = tf.keras.models.Model(inp,out)

In [24]:
NNmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               16640     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0   

In [25]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG151.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [26]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

In [27]:
Nep = 20
NNmodel.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/20
13728/13728 [==============================] - 107s 8ms/step - loss: 565.1663 - val_loss: 429.7754
Epoch 2/20
13728/13728 [==============================] - 107s 8ms/step - loss: 493.7473 - val_loss: 422.5396
Epoch 3/20
13728/13728 [==============================] - 106s 8ms/step - loss: 479.9125 - val_loss: 408.0037
Epoch 4/20
13728/13728 [==============================] - 106s 8ms/step - loss: 473.1182 - val_loss: 399.2956
Epoch 5/20
13728/13728 [==============================] - 107s 8ms/step - loss: 468.0725 - val_loss: 397.7123
Epoch 6/20
13728/13728 [==============================] - 107s 8ms/step - loss: 464.7278 - val_loss: 394.3068
Epoch 7/20
13728/13728 [==============================] - 107s 8ms/step - loss: 461.8383 - val_loss: 390.1633
Epoch 8/20
13728/13728 [==============================] - 107s 8ms/step - loss: 460.0748 - val_loss: 391.1540
Epoch 9/20
13728/13728 [==============================] - 107s 8ms/step - loss: 457.8499 - val_loss: 386.3749
Epoch 10/2

In [28]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/152_POG_84G_p4K_RHinput_FLUXoutput_EPS.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [29]:
2

2